# Plot based recommender system

We have a dataset with lots of metadata on movies. We will build a recommender system based on cosine similarity scores computed for all movies based on their plot descriptions embeddings.

## Loat the dataset

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
# Let's load the data
import pandas as pd
import numpy as np

pd.set_option('display.max_colwidth', 100)

movies=pd.read_csv('/content/drive/MyDrive/Lab 7: Word Embeddings/P7_Word_Embeddings.csv')
movies.head()

,budget,genres,homepage,id,keywords,original_language,original_title,overview,popularity,production_companies,production_countries,release_date,revenue,runtime,spoken_languages,status,tagline,title,vote_average,vote_count
0,237000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""name"": ""Adventure""}, {""id"": 14, ""name"": ""Fantasy""}, {...",http://www.avatarmovie.com/,19995,"[{""id"": 1463, ""name"": ""culture clash""}, {""id"": 2964, ""name"": ""future""}, {""id"": 3386, ""name"": ""sp...",en,Avatar,"In the 22nd century, a paraplegic Marine is dispatched to the moon Pandora on a unique mission, ...",150.437577,"[{""name"": ""Ingenious Film Partners"", ""id"": 289}, {""name"": ""Twentieth Century Fox Film Corporatio...","[{""iso_3166_1"": ""US"", ""name"": ""United States of America""}, {""iso_3166_1"": ""GB"", ""name"": ""United ...",2009-12-10,2787965087,162.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}, {""iso_639_1"": ""es"", ""name"": ""Espa\u00f1ol""}]",Released,Enter the World of Pandora.,Avatar,7.2,11800
1,300000000,"[{""id"": 12, ""name"": ""Adventure""}, {""id"": 14, ""name"": ""Fantasy""}, {""id"": 28, ""name"": ""Action""}]",http://disney.go.com/disneypictures/pirates/,285,"[{""id"": 270, ""name"": ""ocean""}, {""id"": 726, ""name"": ""drug abuse""}, {""id"": 911, ""name"": ""exotic is...",en,Pirates of the Caribbean: At World's End,"Captain Barbossa, long believed to be dead, has come back to life and is headed to the edge of t...",139.082615,"[{""name"": ""Walt Disney Pictures"", ""id"": 2}, {""name"": ""Jerry Bruckheimer Films"", ""id"": 130}, {""na...","[{""iso_3166_1"": ""US"", ""name"": ""United States of America""}]",2007-05-19,961000000,169.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,"At the end of the world, the adventure begins.",Pirates of the Caribbean: At World's End,6.9,4500
2,245000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""name"": ""Adventure""}, {""id"": 80, ""name"": ""Crime""}]",http://www.sonypictures.com/movies/spectre/,206647,"[{""id"": 470, ""name"": ""spy""}, {""id"": 818, ""name"": ""based on novel""}, {""id"": 4289, ""name"": ""secret...",en,Spectre,A cryptic message from Bond’s past sends him on a trail to uncover a sinister organization. Whil...,107.376788,"[{""name"": ""Columbia Pictures"", ""id"": 5}, {""name"": ""Danjaq"", ""id"": 10761}, {""name"": ""B24"", ""id"": ...","[{""iso_3166_1"": ""GB"", ""name"": ""United Kingdom""}, {""iso_3166_1"": ""US"", ""name"": ""United States of ...",2015-10-26,880674609,148.0,"[{""iso_639_1"": ""fr"", ""name"": ""Fran\u00e7ais""}, {""iso_639_1"": ""en"", ""name"": ""English""}, {""iso_639...",Released,A Plan No One Escapes,Spectre,6.3,4466
3,250000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 80, ""name"": ""Crime""}, {""id"": 18, ""name"": ""Drama""}, {""id"": ...",http://www.thedarkknightrises.com/,49026,"[{""id"": 849, ""name"": ""dc comics""}, {""id"": 853, ""name"": ""crime fighter""}, {""id"": 949, ""name"": ""te...",en,The Dark Knight Rises,"Following the death of District Attorney Harvey Dent, Batman assumes responsibility for Dent's c...",112.312950,"[{""name"": ""Legendary Pictures"", ""id"": 923}, {""name"": ""Warner Bros."", ""id"": 6194}, {""name"": ""DC E...","[{""iso_3166_1"": ""US"", ""name"": ""United States of America""}]",2012-07-16,1084939099,165.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,The Legend Ends,The Dark Knight Rises,7.6,9106
4,260000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""name"": ""Adventure""}, {""id"": 878, ""name"": ""Science Fic...",http://movies.disney.com/john-carter,49529,"[{""id"": 818, ""name"": ""based on novel""}, {""id"": 839, ""name"": ""mars""}, {""id"": 1456, ""name"": ""medal...",en,John Carter,"John Carter is a war-weary, former military captain who's inexplicably transported to the myster...",43.926995,"[{""name"": ""Walt Disney Pictures"", ""id"": 2}]","[{""iso_3166_1"": ""US"", ""name"": ""United States of Ameri

In [5]:
# We're only interested in the title and the overview
plot=movies.loc[:,['title','overview']]
plot.head()

,title,overview
0,Avatar,"In the 22nd century, a paraplegic Marine is dispatched to the moon Pandora on a unique mission, ..."
1,Pirates of the Caribbean: At World's End,"Captain Barbossa, long believed to be dead, has come back to life and is headed to the edge of t..."
2,Spectre,A cryptic message from Bond’s past sends him on a trail to uncover a sinister organization. Whil...
3,The Dark Knight Rises,"Following the death of District Attorney Harvey Dent, Batman assumes responsibility for Dent's c..."
4,John Carter,"John Carter is a war-weary, former military captain who's inexplicably transported to the myster..."


In [6]:
print(plot.shape)
plot.isna().sum()

(4803, 2)


title       0
overview    3
dtype: int64

In [7]:
# Let's drop the null values
plot.dropna(inplace=True)

## Clean the dataset

Now let's define a function to perform some pre-processing to the overview column since that's the one we'll transform into features.

In [13]:
import nltk
from nltk import word_tokenize
nltk.download('stopwords')
nltk.download('punkt')
nltk.download('wordnet')
stopwords = nltk.corpus.stopwords.words('english')
wn = nltk.WordNetLemmatizer()

# We tokenize the sentence, remove punctuation,
# remove stopwords and lemmatize.
# Finally, we convert it back to a string.

def clean_text(text):
    tokens = word_tokenize(str(text))
    nopunct = [word.lower() for word in tokens if word.isalpha()]
    nosw = [word for word in nopunct if word not in stopwords]
    lemmas = [wn.lemmatize(word) for word in nosw]
    return(" ".join(lemmas))

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.


In [14]:
plot['clean_overview']=plot['overview'].apply(clean_text)

In [15]:
plot.head()

,title,overview,clean_overview
0,Avatar,"In the 22nd century, a paraplegic Marine is dispatched to the moon Pandora on a unique mission, ...",century paraplegic marine dispatched moon pandora unique mission becomes torn following order pr...
1,Pirates of the Caribbean: At World's End,"Captain Barbossa, long believed to be dead, has come back to life and is headed to the edge of t...",captain barbossa long believed dead come back life headed edge earth turner elizabeth swann noth...
2,Spectre,A cryptic message from Bond’s past sends him on a trail to uncover a sinister organization. Whil...,cryptic message bond past sends trail uncover sinister organization battle political force keep ...
3,The Dark Knight Rises,"Following the death of District Attorney Harvey Dent, Batman assumes responsibility for Dent's c...",following death district attorney harvey dent batman assumes responsibility dent crime protect l...
4,John Carter,"John Carter is a war-weary, former military captain who's inexplicably transported to the myster...",john carter former military captain inexplicably transported mysterious exotic planet barsoom ma...


## Obtain the embeddings
Here, you have several options:
* You can use the BOW approach
* You can use TfIdf embeddings
* You can use pretrained embeddings (Spacy, Fasttext...)
* You can create your own embeddings

Now we'll transform the clean overview to Spacy embeddings.

In [17]:
!python -m spacy download en_core_web_md # Medium model for English

     |████████████████████████████████| 96.4 MB 1.2 MB/s 
  Created wheel for en-core-web-md: filename=en_core_web_md-2.2.5-py3-none-any.whl size=98051301 sha256=7ceb02c707017e7a2f643571645d5c16d24e8a51c6e58a3495f7c8e5616d4038
  Stored in directory: /tmp/pip-ephem-wheel-cache-hv04y7na/wheels/69/c5/b8/4f1c029d89238734311b3269762ab2ee325a42da2ce8edb997
Successfully built en-core-web-md
✔ Download and installation successful
You can now load the model via spacy.load('en_core_web_md')


In [18]:
import en_core_web_md # We import the model 

nlp = en_core_web_md.load() # We create a nlp object

In [19]:
def sent_vector(sentence):
  vector_sent = nlp(sentence).vector
  return (vector_sent)

In [20]:
vec_data = []
for row in plot['clean_overview']:
  vec_data.append(sent_vector(row))

In [21]:
vec_data_df = pd.DataFrame(data=vec_data)

In [22]:
vec_data_df.shape

(4800, 300)

Now we  will compute the Cosine Similarity for the whole matrix.

In [23]:
from sklearn.metrics.pairwise import cosine_similarity

similarity_scores = cosine_similarity(vec_data_df,vec_data_df)

In [24]:
similarity_scores

array([[1.0000007 , 0.7714376 , 0.8300441 , ..., 0.82699084, 0.7499687 ,
        0.7585456 ],
       [0.7714376 , 0.99999946, 0.81167614, ..., 0.80811036, 0.8013756 ,
        0.85801035],
       [0.8300441 , 0.81167614, 0.99999976, ..., 0.87920725, 0.8129188 ,
        0.7971883 ],
       ...,
       [0.82699096, 0.8081107 , 0.8792072 , ..., 0.9999998 , 0.89147854,
        0.85573465],
       [0.7499688 , 0.8013754 , 0.81291884, ..., 0.89147854, 0.9999999 ,
        0.86318344],
       [0.75854534, 0.85801035, 0.7971883 , ..., 0.85573465, 0.86318344,
        0.9999998 ]], dtype=float32)

In [25]:
#Now let's store the title and the indices
indices = pd.Series(plot.index, index=plot['title'])
indices.shape
indices.head()

title
Avatar                                      0
Pirates of the Caribbean: At World's End    1
Spectre                                     2
The Dark Knight Rises                       3
John Carter                                 4
dtype: int64

## Create the recommender function

We are now in a good position to define our recommendation function. These are the following steps we'll follow:
* Get the index of the movie given its title.
* Get the list of cosine similarity scores for that particular movie with all movies.
* Convert it into a list of tuples where the first element is its position and the second is the similarity score.
* Sort the aforementioned list of tuples based on the similarity scores; that is, the second element.
* Get the top 10 elements of this list. Ignore the first element as it refers to self (the movie most similar to a particular movie is the movie itself).
* Return the titles corresponding to the indices of the top elements.

In [26]:
# Function that takes in movie title as input and outputs most similar movies

def get_recommendations(title):
    
    # Check if the movie is contained in the dataset
    if title in indices.keys():
        # Get the index of the movie that matches the title
        idx = indices[title]

        # Get the cosine similarity scores of all movies with that movie
        sim_scores = list(enumerate(similarity_scores[idx]))

        # Sort the movies based on the similarity scores
        sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)

        # Get the scores of the 10 most similar movies
        sim_scores = sim_scores[1:11]

        # Get the movie indices
        movie_indices = [i[0] for i in sim_scores]

        # Return the top 10 most similar movies
        top_10_titles = plot['title'].iloc[movie_indices]
        top_10_overviews = plot['overview'].iloc[movie_indices]
        return pd.DataFrame([top_10_titles, top_10_overviews]).transpose()
    else:
        print("This movie is not included in our dataset, sorry. Try again.")
        return

In [27]:
get_recommendations('The Dark Knight Rises')

,title,overview
65,The Dark Knight,Batman raises the stakes in his war on crime. With the help of Lt. Jim Gordon and District Attor...
299,Batman Forever,The Dark Knight of Gotham City confronts a dastardly duo: Two-Face and the Riddler. Formerly Dis...
119,Batman Begins,"Driven by tragedy, billionaire Bruce Wayne dedicates his life to uncovering and defeating the co..."
85,Captain America: The Winter Soldier,"After the cataclysmic events in New York with The Avengers, Steve Rogers, aka Captain America is..."
4644,Teeth and Blood,"A beautiful diva is murdered on the set of horror director Vincent Augustine's latest film ""Chap..."
1742,Brick Mansions,"In a dystopian Detroit, grand houses that once housed the wealthy are now homes of the city's mo..."
238,Teenage Mutant Ninja Turtles,The city needs heroes. Darkness has settled over New York City as Shredder and his evil Foot Cla...
1103,The Fugitive,"Wrongfully accused of murdering his wife, Richard Kimble escapes from the law in an attempt to f..."
1359,Batman,The Dark Knight of Gotham City begins his war on crime with his first major enemy being the clow...
1084,The Glimmer Man,"A former government operative renowned for his stealth, Jack Cole is now a Los Angeles police de..."


In [28]:
get_recommendations('Avatar')

,title,overview
1959,Star Trek IV: The Voyage Home,"Fugitives of the Federation for their daring rescue of Spock from the doomed Genesis Planet, Adm..."
228,Oblivion,Jack Harper is one of the last few drone repairmen stationed on Earth. Part of a massive operat...
3158,Alien,"During its return to the earth, commercial spaceship Nostromo intercepts a distress signal from ..."
2696,Jason X,"In the year 2455, Old Earth is now a contaminated planet abandoned for centuries -- a brown worl..."
91,Independence Day: Resurgence,"We always knew they were coming back. Using recovered alien technology, the nations of Earth hav..."
2900,Space Battleship Yamato,"In 2199, five years after the Gamilons began an invasion of Earth, the planet has been ravaged b..."
27,Battleship,"When mankind beams a radio signal into space, a reply comes from ‘Planet G’, in the form of seve..."
778,Meet Dave,A crew of miniature aliens operate a spaceship that has a human form. While trying to save their...
41,Green Lantern,"For centuries, a small but powerful force of warriors called the Green Lantern Corps has sworn t..."
270,The Martian,"During a manned mission to Mars, Astronaut Mark Watney is presumed dead after a fierce storm and..."
